# Final Training and Submission

Using BEiT Large model + the whole train dataset

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from keras.applications import imagenet_utils

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
DATASET_SIZE = 9367
IMAGE_SIZE = 224
BATCH_SIZE = 16
WORKERS = 4
EPOCHS = 10

BASE_PATH='../data'

classes = [
    'cup', 
    'fork', 
    'glass', 
    'knife', 
    'plate', 
    'spoon'
]

First, we will load the training dataframe and split it into train and validation

In [3]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

,Id,label,filename
0,0560,glass,data/images/0560.jpg
1,4675,cup,data/images/4675.jpg
2,0875,glass,data/images/0875.jpg
3,4436,spoon,data/images/4436.jpg
4,8265,plate,data/images/8265.jpg


In [4]:
# val_cutoff = int(len(df_train_full) * 0.8)
# df_train = df_train_full[:val_cutoff]
# df_val = df_train_full[val_cutoff:]

Now let's create image generators

In [5]:
# These models don't have the imagenet preprocessing built in so I have to apply this
def preprocess_input(x, data_format=None):
    return imagenet_utils.preprocess_input(
        x, data_format=data_format, mode="tf"
    )

In [6]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train_full,
    x_col='filename',
    y_col='label',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
)

Found 5559 validated image filenames belonging to 6 classes.


In [7]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

array(['cup', 'fork', 'glass', 'knife', 'plate', 'spoon'], dtype='<U5')

I adjust early stopping to end sooner (2 epochs and min delta of 1e-2) as a form of regularization, as I no longer have a validation set.

In [8]:
earlystopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'accuracy',
    min_delta = 1e-2,
    patience = 2,
    mode = 'max',
    restore_best_weights = False,
    verbose = 1
)

callbacks = [earlystopping]

In [9]:
from keras_cv_attention_models import beit

In [10]:
base_model = beit.BeitLargePatch16( #BeitBasePatch16(
    pretrained='imagenet21k-ft1k',  #weights='imagenet',
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    num_classes=len(classes)
)
base_model.trainable = False

>>>> Load pretrained from: C:\Users\andre\.keras\models\beit_large_patch16_224_imagenet21k-ft1k.h5


In [11]:
base_model = tf.keras.Model(inputs=base_model.layers[1].input, outputs= base_model.layers[-3].output)

In [12]:
inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

model = tf.keras.Sequential([
    inputs,
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(12, 'gelu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(6, 'softmax')
],
name = 'beitL16_224')

In [13]:
learning_rate = 0.003
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [14]:
history = model.fit(
    x = train_generator,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    workers=WORKERS,
    callbacks=callbacks
)

Epoch 1/10
348/348 [==============================] - 121s 291ms/step - loss: 0.1765 - accuracy: 0.9572
Epoch 2/10
348/348 [==============================] - 102s 292ms/step - loss: 0.0646 - accuracy: 0.9824
Epoch 3/10
348/348 [==============================] - 102s 292ms/step - loss: 0.0508 - accuracy: 0.9861
Epoch 4/10
348/348 [==============================] - 101s 291ms/step - loss: 0.0460 - accuracy: 0.9849
Epoch 4: early stopping


Now let's use this model to predict the labels for test data

In [15]:
df_test = pd.read_csv('data/test.csv', dtype={'Id': str})
df_test['filename'] = 'data/images/' + df_test['Id'] + '.jpg'
df_test.head()

,Id,filename
0,0678,data/images/0678.jpg
1,3962,data/images/3962.jpg
2,9271,data/images/9271.jpg
3,5133,data/images/5133.jpg
4,8842,data/images/8842.jpg


In [19]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 3808 validated image filenames.


In [20]:
y_pred = model.predict(test_generator)

238/238 [==============================] - 4761s 20s/step


In [21]:
predictions = classes[y_pred.argmax(axis=1)]

Finally, we need to prepare the submission

In [22]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len('data/images/'):-4]
del df_submission['filename']

In [23]:
df_submission[['Id', 'label']].to_csv('submission.csv', index=False)

In [5]:
!kaggle competitions submit kitchenware-classification -f submission.csv -m "No validation"

Successfully submitted to Kitchenware Classification



  0%|          | 0.00/42.5k [00:00<?, ?B/s]
 19%|█▉        | 8.00k/42.5k [00:00<00:00, 71.4kB/s]
100%|██████████| 42.5k/42.5k [00:00<00:00, 59.0kB/s]
